In [ ]:
#!/usr/bin/env python
# encoding: utf-8

In [ ]:
import csv
import pandas as pd 
import numpy as np 
from collections import defaultdict
import re
import sys
from sklearn.model_selection import train_test_split
import datetime
import matplotlib.pyplot as plt
import time
#import numpy as np
from pprint import pprint

import spacy  # For preprocessing
import nltk
import string
from nltk.corpus import stopwords
import preprocessor as p  #pip install tweet-preprocessor
import logging  # Setting up the loggings to monitor gensim
from nltk.stem import WordNetLemmatizer
from string import punctuation as punc

import gensim
import gensim.corpora as corpora
#from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import gensim.models as gsm
from gensim.test.utils import datapath

import regex
import emoji #pip install emoji --upgrade
# Internal dependencies
import word_emoji2vec as we2v
#from word_emoji2vec import Word_Emoji2Vec

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd
import torchvision.transforms as transforms
from torch.autograd import Variable

####BERT

from transformers import BertTokenizer, BertModel

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

nlp = spacy.load('en', disable=['ner', 'parser']) # disabling Named Entity Recognition for speed #python -m spacy download en

torch.manual_seed(1)
torch.cuda.manual_seed(1)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.enabled = False
np.random.seed(1)

# torch.manual_seed(1)
# np.random.seed(1)

In [ ]:
# text = "Here is the sentence I want embeddings for The paper 😘 lol verrry"
# marked_text = "[CLS] " + text + " [SEP]"

# # Tokenize our sentence with the BERT tokenizer.
# tokenized_text = tokenizer.tokenize(marked_text)

# # Print out the tokens.
# print (tokenized_text)

In [ ]:
# list(tokenizer.vocab.keys())[5000:5020]

In [ ]:
df = pd.read_csv("data/yoga_user_name_loc_des_mergetweets_yoga_1300_lb.csv") 
df.sample(10)

In [ ]:
##### load train users and split into train and validation #######
import random
with open("data/train.txt", "r") as f:
    data = f.read().split('\n')
random.seed(1)
random.shuffle(data)

train_data = data[:830] #80% train  
#print(train_data, len(train_data)) #830
valid_data = data[830:] #20% validation
#print(valid_data)


In [ ]:
####.........Create dataframe for Train data
name = []
location = []
description = []
text = []
utype = []
umotivation = []
for i in range (0,len(train_data)):
    for j in range (0,df.shape[0]):
        if df.name[j] == train_data[i]:
            name.append(df.name[j])
            location.append(df.location[j])
            description.append(df.description[j])
            text.append(df.text[j])
            utype.append(df.utype[j])
            umotivation.append(df.umotivation[j])
            break


df_train = pd.DataFrame(data= name, columns=['Name'])
df_train['location'] = location
df_train['description'] = description   
df_train['text'] = text
df_train['utype'] = utype  
df_train['umotivation'] = umotivation   


In [ ]:
df_train 

In [ ]:
####.........Create dataframe for validation data
name = []
location = []
description = []
text = []
utype = []
umotivation = []
for i in range (0,len(valid_data)):
    for j in range (0,df.shape[0]):
        if df.name[j] == valid_data[i]:
            name.append(df.name[j])
            location.append(df.location[j])
            description.append(df.description[j])
            text.append(df.text[j])
            utype.append(df.utype[j])
            umotivation.append(df.umotivation[j])
            break


df_valid = pd.DataFrame(data= name, columns=['Name'])
df_valid['location'] = location
df_valid['description'] = description   
df_valid['text'] = text
df_valid['utype'] = utype  
df_valid['umotivation'] = umotivation   


In [ ]:
df_valid

In [ ]:
########.....Load Test data.......
with open("data/test.txt", "r") as f:
    data = f.read().split('\n')
test_data = data[:] 
#print(test_data, len(test_data)) #262

In [ ]:
####.........Create dataframe for Test data
name = []
location = []
description = []
text = []
utype = []
umotivation = []
for i in range (0,len(test_data)):
    for j in range (0,df.shape[0]):
        if df.name[j] == test_data[i]:
            name.append(df.name[j])
            location.append(df.location[j])
            description.append(df.description[j])
            text.append(df.text[j])
            utype.append(df.utype[j])
            umotivation.append(df.umotivation[j])
            break


df_test = pd.DataFrame(data= name, columns=['Name'])
df_test['location'] = location
df_test['description'] = description   
df_test['text'] = text
df_test['utype'] = utype  
df_test['umotivation'] = umotivation   


In [ ]:
df_test

In [ ]:
###### text preprocessing #########

def remove_string_noise(input_str):
    input_str = re.sub(r"http\S+", "", input_str)
    
    #give special char you want to remove
    #do not put space between chars, and space (" ") is not a special char
    punctuation_noise ="!\"$%&'#()*+,-./:;<=>?@[\]^_`{|}~" #print string.punctuation 
    #number_noise = "0123456789"
    special_noise = ""

    #all_noise = punctuation_noise + number_noise + special_noise
    all_noise = punctuation_noise + special_noise

    for c in all_noise:
        if c in input_str:
            input_str = input_str.replace(c, " ")#replace with space
    fresh_str = ' '.join(input_str.split())
    return fresh_str

def clean_tweets(tweet):
#     #HappyEmoticons
#     emoticons_happy = set([
#         ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
#         ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
#         '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
#         'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
#         '<3'
#         ])

#     # Sad Emoticons
#     emoticons_sad = set([
#         ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
#         ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
#         ':c', ':{', '>:\\', ';('
#         ])

#     #combine sad and happy emoticons
#     emoticons = emoticons_happy.union(emoticons_sad)

    wordnet_lemmatizer = WordNetLemmatizer()
    sw = set(stopwords.words('english'))
    operators = set(('no', 'not', 'nor', 'none'))
    stop_words = set(sw) - operators
    stop_words.update([ 'amp', 'rt'])  ###as we are using set so we used .update....otherwise .extends
    
#     tweet = re.sub(r':', '', tweet)
#     tweet = re.sub(r'‚Ä¶', '', tweet)
#     tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)
    word_tokens = nltk.word_tokenize(tweet)
    filtered_tweet = []
    for w in word_tokens:
        #if w not in stop_words and w not in emoticons:
        if w not in stop_words:
            filtered_tweet.append(w)
    lemmatized_tweet = []
    for word in filtered_tweet:
        lemmatized_tweet.append(wordnet_lemmatizer.lemmatize(word, pos="v"))

    #print (sentences)
    #return ' '.join(filtered_tweet)
    return ' '.join(lemmatized_tweet)
    #return ' '.join(sentences)
    
def pre_processing_tweets (df):
    #p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.SMILEY, p.OPT.MENTION, p.OPT.RESERVED)
    p.set_options(p.OPT.RESERVED)
    
    clean_text = []
    #for i in range (0, df.shape[0]):
        #clean_text.append(p.clean(str(df['text'][i])))#python 3

    for i in df:
        clean_text.append(p.clean(str(i)))#python 3
    #print(clean_text) #all ok
    fresh_text1 = []
    for i in range (0, df.shape[0]):
        #fresh_text1.append(remove_string_noise(clean_text[i].encode('ascii', 'ignore').decode("utf-8"))) #can remove other emojis and no \UF..
        fresh_text1.append(remove_string_noise(clean_text[i]))
    #print(fresh_text1) #all ok
    
    #Call clean_tweet method-2 for extra preprocessing
    filtered_tweet = []
    for i in range (0, len(fresh_text1)):
        filtered_tweet.append(clean_tweets(fresh_text1[i].lower()))

#     #####for bigram
    
#     df_clean = pd.DataFrame({'clean': filtered_tweet})
#     #df_clean = df_clean.dropna().drop_duplicates()
#     sent = [row.split() for row in df_clean['clean']]
#     #print (sent_word, type(sent_word))

#     ## create bigram
#     #Creates the relevant phrases from the list of sentences:
#     phrases = Phrases(sent, min_count=1, progress_per=10000)
#     #print (phrases, type(phrases))

#     #The goal of Phraser() is to cut down memory consumption of Phrases(), by discarding model state not strictly needed for the bigram detection task:
#     bigram = Phraser(phrases)
#     #print (bigram)

#     #Transform the corpus based on the bigrams detected:
#     sentences = bigram[sent]
#     training_data  = [sentences[i] for i in range(0,len(sentences))]
#     return training_data
    
    return filtered_tweet
    

In [ ]:
sentences_train = pre_processing_tweets(df_train.location)
sentences_valid = pre_processing_tweets(df_valid.location)
sentences_test = pre_processing_tweets(df_test.location)

In [ ]:
# Get the lists of sentences and their labels.

sentences_train = np.array(sentences_train)
labels_train = torch.tensor(df_train.umotivation.values)
print(type(sentences_train))

sentences_valid = np.array(sentences_valid)
labels_valid = torch.tensor(df_valid.umotivation.values)
print(type(sentences_valid))

sentences_test = np.array(sentences_test)
labels_test = torch.tensor(df_test.umotivation.values)
print(type(sentences_test))

In [ ]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [ ]:
# # Print the original sentence.
# print(' Original: ', sentences[0])

# # Print the sentence split into tokens.
# print('Tokenized: ', tokenizer.tokenize(sentences[0]))

# # Print the sentence mapped to token ids.
# print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

In [ ]:
max_len = 0

# For every sentence...
for sent in sentences_train:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length train: ', max_len) # 347978

In [ ]:
max_len = 0

# For every sentence...
for sent in sentences_valid:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length valid: ', max_len) # 347978

In [ ]:
max_len = 0

# For every sentence...
for sent in sentences_test:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length test: ', max_len) # 347978

In [ ]:
#The first four features are in tokenizer.encode, but I’m using tokenizer.encode_plus to get the fifth item (attention masks).

# Tokenize all of the sentences and map the tokens to thier word IDs.
def token_id(sentences):
    input_ids = []
    attention_masks = []

    # For every sentence...
    for sent in sentences:
        # `encode_plus` will:
        #   (1) Tokenize the sentence.
        #   (2) Prepend the `[CLS]` token to the start.
        #   (3) Append the `[SEP]` token to the end.
        #   (4) Map tokens to their IDs.
        #   (5) Pad or truncate the sentence to `max_length`
        #   (6) Create attention masks for [PAD] tokens.
        encoded_dict = tokenizer.encode_plus(
                            sent,                      # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = 160,           # Pad & truncate all sentences.
                            pad_to_max_length = True,
                            return_attention_mask = True,   # Construct attn. masks.
                            truncation=True, #explicitely truncate examples to max length. #my add
                            return_tensors = 'pt',     # Return pytorch tensors.
                       )

        # Add the encoded sentence to the list.    
        input_ids.append(encoded_dict['input_ids'])

        # And its attention mask (simply differentiates padding from non-padding).
        attention_masks.append(encoded_dict['attention_mask'])

    # Convert the lists into tensors.
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    #labels = torch.tensor(labels)

#     # Print sentence 0, now as a list of IDs.
#     print('Original: ', sentences_train[0])
#     print('Token IDs:', input_ids_train[0])
    return input_ids, attention_masks


In [ ]:
input_ids_train, attention_masks_train = token_id(sentences_train )
input_ids_valid, attention_masks_valid = token_id(sentences_valid )
input_ids_test, attention_masks_test = token_id(sentences_test)

In [ ]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
train_dataset = TensorDataset(input_ids_train, attention_masks_train, labels_train)
val_dataset = TensorDataset(input_ids_valid, attention_masks_valid, labels_valid)
test_dataset = TensorDataset(input_ids_test, attention_masks_test, labels_test)

In [ ]:
##...BERT Fine Tuning tutotrial

In [ ]:
import torch
# torch.cuda.empty_cache() 
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    #device = torch.device("cuda") # select the zeroth GPU with this line: gpu = 0
    #device = torch.cuda.set_device(1)  #wrong provide device = None  
    device = torch.device(1) #(use cuda device 1) for gpu = 1
    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name())

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

#device = torch.device("cpu") # uncomment for cpu use

In [ ]:
#print ('Current cuda device ', torch.cuda.current_device())
print ('Current cuda device ', device)

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32 #good
#batch_size = 16
# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )



In [ ]:
from transformers import BertTokenizer, BertModel

bert = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
import torch.nn as nn
from torchnlp.nn import Attention #pip imstall pytorch-nlp
class BERTLSTMSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size'] #768
              
        self.rnn = nn.LSTM(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        
        
        
        self.dropout = nn.Dropout(dropout)
        self.attn_fc = Attention(hidden_dim * 2 if bidirectional else hidden_dim) #attention layer from torchnlp
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        #self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, hidden_dim)
        #self.fc1 = nn.Linear(150, output_dim) 
        #self.fc1 = nn.Linear(hidden_dim, output_dim) 
        #self.fc2 = nn.Linear(50, output_dim)

    def forward(self, text):
        
        #text = [batch size, sent len]
        #print("text", text.size()) #torch.Size([32, 50])       
        with torch.no_grad():
            embedded = self.bert(text)[0]
                
        #embedded = [batch size, sent len, emb dim]
        #print("embedded", type(embedded), embedded.size()) #torch.Size([32, 50, 768])
        #embedded = self.dropout(embedded) #add dropout...don't use it...dropout layer after an pre-trained embedding, that the weights are not learnt during training
        lstm_out, (hidden, c_n) = self.rnn(embedded) #for lstm
        
        #hidden = [n layers * n directions, batch size, hid dim]
        #print("hidden before", type(hidden), hidden.size()) #torch.Size([4, 32, 256])
        if self.rnn.bidirectional: #add dropout
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
        
#         if self.rnn.bidirectional: #No dropout
#             hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
#         else:
#             hidden = hidden[-1,:,:]
            
        #lstm_out = self.dropout(lstm_out) #add dropout
    
        #hidden = [batch size, hid dim]
        #print("hidden after", type(hidden), hidden.size()) #torch.Size([32, 512]) cz of bi-direction = 2 * 256 = 512
        #attn_out = self.attention(lstm_out, hidden)
#         query = hidden = [batch size, output length, dimensions]
#         context = lstm_out = [batch size, query length, dimensions]
#        input for attention attention(query, context)
#       output for attention tuple with output and weights
        #attn_out = self.attn_fc(hidden.unsqueeze(1), lstm_out)
        #print(type(attn_out[0]), attn_out[0].size(), type(attn_out[1]), attn_out[1].size()) #output = torch.Size([32, 1, 512]), weight = torch.Size([32, 1, 160])
        #print("attn_out", attn_out.size())
        output = self.out(hidden)
        
        #output = self.out(attn_out[0].squeeze(1))
        
        #output = self.fc1(output)
        #output = self.fc2(output)
        #output = [batch size, out dim]
        #print("output", type(output), output.size()) #torch.Size([32, 3])
        return output

In [ ]:
#HIDDEN_DIM = 256 #original
HIDDEN_DIM = 300 #good
OUTPUT_DIM = 3
N_LAYERS = 2 #original good
#N_LAYERS = 3
BIDIRECTIONAL = True
#DROPOUT = 0.25 #original
DROPOUT = 0.5 #good


model = BERTLSTMSentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)

In [ ]:
# def count_parameters(model):
#     return sum(p.numel() for p in model.parameters() if p.requires_grad)

# print(f'The model has {count_parameters(model):,} trainable parameters')

In [ ]:
###In order to freeze paramers (not train them) we need to set their requires_grad attribute to False. To do this, we simply loop through all of the named_parameters in our model and if they're a part of the bert transformer model, we set requires_grad = False.

for name, param in model.named_parameters():    
    #print(name)
    if name.startswith('bert'):
        param.requires_grad = False

In [ ]:
# for name, param in model.named_parameters():                
#     if param.requires_grad:
#         print(name)

In [ ]:
import torch.optim as optim
#learning_rate=0.01
optimizer = optim.Adam(model.parameters()) #original #good #f1 = 64%
#optimizer = optim.Adadelta(model.parameters())
#optimizer = optim.Adadelta(model.parameters(), lr=learning_rate, eps=1e-06, weight_decay=0.0001)
#optimizer = optim.Adam(model.parameters(), lr = learning_rate, weight_decay=0.0001)
#optimizer = optim.SGD(model.parameters(), lr = learning_rate, momentum = 0.9, weight_decay=0) #f1 = 68%, lr = .01

In [ ]:
#criterion = nn.BCEWithLogitsLoss()
criterion = nn.CrossEntropyLoss()

In [ ]:
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    #rounded_preds = torch.round(torch.sigmoid(preds))
    #rounded_preds = torch.round(preds)
    correct = (preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    
    from sklearn.metrics import f1_score
    macro_f1 = f1_score(y.to("cpu"), preds.to("cpu"), average='macro')
    #print("macro_f1", macro_f1)

    return acc, macro_f1

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    epoch_macro = 0
    model.train()
    for step, batch in enumerate(iterator):    
    #for batch in iterator:
        #print("batch", batch)
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
#         b_input_ids = batch[0].to(device)
#         b_input_mask = batch[1].to(device)
#         b_labels = batch[2].to(device)

        optimizer.zero_grad()
        text = batch[0].to(device)
        label = batch[2].to(device)
        #print("label", label, type(label),label.size()) #torch.Size([32])
        #label = label.unsqueeze(1)
        #print("label", label, type(label),label.size())
        #predictions = model(batch.text).squeeze(1)
        predictions = model(text)
        #pred = model(text)
        #print("pred", pred.size()) #torch.Size([32, 3])
        #predictions = torch.argmax(pred, dim = 1)
        #print("predictions", predictions, predictions.size()) #torch.Size([32, 3])
        #loss = criterion(predictions, batch.label)
        loss = criterion(predictions, label)
        #print("loss", loss)
        #acc = binary_accuracy(predictions, batch.label)
        acc, macro_f1 = binary_accuracy(torch.argmax(predictions, dim = 1), label)
        #print("acc", acc)
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        epoch_macro += macro_f1.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator), epoch_macro / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    epoch_macro = 0
    model.eval()
    
    with torch.no_grad():
    
        #for batch in iterator:
        for step, batch in enumerate(iterator):    
            text = batch[0].to(device)
            label = batch[2].to(device)

            #predictions = model(batch.text).squeeze(1)
            predictions = model(text)
            #loss = criterion(predictions, batch.label)
            loss = criterion(predictions, label)
            #acc = binary_accuracy(predictions, batch.label)
            acc, macro_f1 = binary_accuracy(torch.argmax(predictions, dim = 1), label)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
            epoch_macro += macro_f1.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator), epoch_macro / len(iterator)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
###Start training
per_epoch_train_loss = []
per_epoch_val_loss = []
per_epoch_train_f1 = []
per_epoch_val_f1 = []
per_epoch_train_acc = []
per_epoch_val_acc = []
N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc, train_f1 = train(model, train_dataloader, optimizer, criterion)
    valid_loss, valid_acc, valid_f1 = evaluate(model, validation_dataloader, criterion)
        
    end_time = time.time()
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    per_epoch_train_loss.append(train_loss)
    per_epoch_val_loss.append(valid_loss)
    per_epoch_train_f1.append(train_f1)
    per_epoch_val_f1.append(valid_f1)
    per_epoch_train_acc.append(train_acc)
    per_epoch_val_acc.append(valid_acc)
    
    if valid_loss <= best_valid_loss:
        best_valid_loss = valid_loss
        print("best model saved in epoch :", epoch+1 )
    torch.save(model.state_dict(), 'data/bert_loc/model_loc_umotivation_wo_attn_'+str(epoch+1)+'.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f} | Train macro-avg-f1: {train_f1*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f} |  Val. macro-avg-f1: {valid_f1*100:.2f}%')
    
    
    

In [ ]:
def save_plots(train_losses, val_losses, train_f1, val_f1, train_accs, val_accs):
    """Plot

        Plot two figures: loss vs. epoch and accuracy vs. epoch
    """
    n = len(train_losses)
    xs = np.arange(1,n+1,1)

    # plot train and val losses
    fig, ax = plt.subplots()
    ax.plot(xs, train_losses, '--', linewidth=2, label='train loss')
    ax.plot(xs, val_losses, '-', linewidth=2, label='validation loss')
    #ax.set_xlim(0, 10)
    ax.set_xlabel("Epoch")
    ax.set_ylabel("Loss")
    ax.legend(loc='upper right')
    plt.savefig('data/bert_plot/loss_Bert_Loc_umotivation_wo_attn.png')

    # plot train and val f1-score
    #plt.clf()
    fig, ax = plt.subplots()
    ax.plot(xs, train_f1, '--', linewidth=2, label='train')
    ax.plot(xs, val_f1, '-', linewidth=2, label='validation')
    
    ax.set_xlabel("Epoch")
    ax.set_ylabel("Macro-avg F1")
    ax.legend(loc='lower right')
    plt.savefig('data/bert_plot/f1_Bert_Loc_umotivation_wo_attn.png')
    
    # plot train and val accuracy
    #plt.clf()
    fig, ax = plt.subplots()
    ax.plot(xs, train_accs, '--', linewidth=2, label='train')
    ax.plot(xs, val_accs, '-', linewidth=2, label='validation')
    #ax.set_xlim(0, 10)
    ax.set_xlabel("Epoch")
    ax.set_ylabel("Accuracy F1")
    ax.legend(loc='lower right')
    plt.savefig('data/bert_plot/acc_Bert_Loc_umotivation_wo_attn.png')
    
save_plots(per_epoch_train_loss, per_epoch_val_loss, per_epoch_train_f1, per_epoch_val_f1, per_epoch_train_acc, per_epoch_val_acc)

In [ ]:
# For Test the order doesn't matter, so we'll just read them sequentially.
test_dataloader = DataLoader(
            test_dataset, # The validation samples.
            sampler = SequentialSampler(test_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [ ]:
for epoch in range(0,10):
    model.load_state_dict(torch.load('data/bert_loc/model_loc_umotivation_wo_attn_'+str(epoch+1)+'.pt'))

    test_loss, test_acc, test_f1 = evaluate(model, test_dataloader, criterion)

    print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f} | Test macro-avg-f1: {test_f1*100:.2f}%')
